1. Get the HTML

In [1]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd 
df_list = pd.read_html('https://en.wikipedia.org/wiki/Mitch_McConnell')

2.Get the infobox 

The CSS selector of the infobox is class = infobox vcard.

In [3]:
import requests
from bs4 import BeautifulSoup

response = requests.get("https://en.wikipedia.org/wiki/Mitch_McConnell")
content = response.content
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
infobox = soup.find_all(class_="infobox")

3. Make a dataframe

In [5]:
all_keys = [a.text for a in soup.find_all(class_ ="infobox-label")]
all_keys.insert(0, 'Full Name')

full_name = [a.text for a in soup.find_all(class_ = "fn")]
all_values = [a.text for a in soup.find_all(class_ ="infobox-data")]
all_values.insert(0, full_name[0])

In [6]:
all_keys[0:5]

['Full Name', 'Whip', 'Preceded by', 'Whip', 'Preceded by']

In [7]:
all_values[0:5]

['Mitch McConnell',
 'John Thune',
 'Chuck Schumer',
 'Trent LottJon KylJohn Cornyn',
 'Harry Reid']

In [8]:
df = pd.DataFrame({'key': all_keys, 'value':all_values})
df.head()

,key,value
0,Full Name,Mitch McConnell
1,Whip,John Thune
2,Preceded by,Chuck Schumer
3,Whip,Trent LottJon KylJohn Cornyn
4,Preceded by,Harry Reid


In [9]:
newdf = df[df.key.isin(["Full Name", "Political party","Children"])]

In [10]:
newdf

,key,value
0,Full Name,Mitch McConnell
24,Political party,Republican
26,Children,3


4. Make a function

In [11]:
def get_wiki_info(url):
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    all_keys = [a.text for a in soup.find_all(class_ ="infobox-label")]
    all_keys.insert(0, 'Full Name')
    
    full_name = [a.text for a in soup.find_all(class_ = "fn")]
    all_values = [a.text for a in soup.find_all(class_ ="infobox-data")]
    all_values.insert(0, full_name[0])
    
    if "Children" in all_keys:
        pass
    else:
        all_keys.append('Children')
        all_values.append('NA')
        
    df = pd.DataFrame({'key': all_keys, 'value':all_values})
    newdf = df[df.key.isin(["Full Name", "Political party","Children"])]
    return newdf

In [12]:
get_wiki_info('https://en.wikipedia.org/wiki/Tammy_Duckworth')

,key,value
0,Full Name,Tammy Duckworth
13,Political party,Democratic
15,Children,2


In [13]:
get_wiki_info('https://en.wikipedia.org/wiki/Susan_Collins')

,key,value
0,Full Name,Susan Collins
7,Political party,Republican
13,Children,NA


5. Get all senators' pages

In [14]:
all_request = requests.get("https://en.wikipedia.org/wiki/List_of_current_United_States_senators")
all_content = all_request.content
all_soup = BeautifulSoup(all_request.content, 'html.parser')

In [15]:
vcard = all_soup.find_all(class_="vcard")

In [16]:
all_senators = [a.a.get('href') for a in all_soup.find_all(class_="fn")]

In [17]:
all_senators

['/wiki/Kamala_Harris',
 '/wiki/Patrick_Leahy',
 '/wiki/Chuck_Schumer',
 '/wiki/Dick_Durbin',
 '/wiki/Patty_Murray',
 '/wiki/Debbie_Stabenow',
 '/wiki/Mark_Warner',
 '/wiki/Elizabeth_Warren',
 '/wiki/Amy_Klobuchar',
 '/wiki/Bernie_Sanders',
 '/wiki/Joe_Manchin',
 '/wiki/Cory_Booker',
 '/wiki/Tammy_Baldwin',
 '/wiki/Gary_Peters',
 '/wiki/Catherine_Cortez_Masto',
 '/wiki/Jeff_Merkley',
 '/wiki/Brian_Schatz',
 '/wiki/Mitch_McConnell',
 '/wiki/John_Thune',
 '/wiki/John_Barrasso',
 '/wiki/Roy_Blunt',
 '/wiki/Joni_Ernst',
 '/wiki/Rick_Scott',
 '/wiki/Mike_Lee_(American_politician)',
 '/wiki/Mike_Crapo',
 '/wiki/Chuck_Grassley',
 '/wiki/Richard_Shelby',
 '/wiki/Tommy_Tuberville',
 '/wiki/Lisa_Murkowski',
 '/wiki/Dan_Sullivan_(U.S._senator)',
 '/wiki/Kyrsten_Sinema',
 '/wiki/Mark_Kelly',
 '/wiki/John_Boozman',
 '/wiki/Tom_Cotton',
 '/wiki/Dianne_Feinstein',
 '/wiki/Alex_Padilla',
 '/wiki/Michael_Bennet',
 '/wiki/John_Hickenlooper',
 '/wiki/Richard_Blumenthal',
 '/wiki/Chris_Murphy',
 '/wiki/To

In [18]:
all_senators_links = ['https://en.wikipedia.org'+ i for i in all_senators]

In [19]:
all_senators_links

['https://en.wikipedia.org/wiki/Kamala_Harris',
 'https://en.wikipedia.org/wiki/Patrick_Leahy',
 'https://en.wikipedia.org/wiki/Chuck_Schumer',
 'https://en.wikipedia.org/wiki/Dick_Durbin',
 'https://en.wikipedia.org/wiki/Patty_Murray',
 'https://en.wikipedia.org/wiki/Debbie_Stabenow',
 'https://en.wikipedia.org/wiki/Mark_Warner',
 'https://en.wikipedia.org/wiki/Elizabeth_Warren',
 'https://en.wikipedia.org/wiki/Amy_Klobuchar',
 'https://en.wikipedia.org/wiki/Bernie_Sanders',
 'https://en.wikipedia.org/wiki/Joe_Manchin',
 'https://en.wikipedia.org/wiki/Cory_Booker',
 'https://en.wikipedia.org/wiki/Tammy_Baldwin',
 'https://en.wikipedia.org/wiki/Gary_Peters',
 'https://en.wikipedia.org/wiki/Catherine_Cortez_Masto',
 'https://en.wikipedia.org/wiki/Jeff_Merkley',
 'https://en.wikipedia.org/wiki/Brian_Schatz',
 'https://en.wikipedia.org/wiki/Mitch_McConnell',
 'https://en.wikipedia.org/wiki/John_Thune',
 'https://en.wikipedia.org/wiki/John_Barrasso',
 'https://en.wikipedia.org/wiki/Roy_Blu

Average Number of Children

In [20]:
all_senators = []
for i in all_senators_links:
    senator = get_wiki_info(i)
    all_senators.append(senator)
all_senators = pd.concat(all_senators)

In [21]:
all_children = all_senators.loc[all_senators['key'] == 'Children']
children=all_children.fillna(0)

In [22]:
children['number']=children['value'].str.extract(r"^(\d+)")
children_new=children.fillna(0)
children_new['Children Number'] = children_new['number'].astype(int)
children_new.head()

,key,value,number,Children Number
20,Children,NA,0,0
21,Children,3,3,3
28,Children,2,2,2
19,Children,3[note 1],3,3
21,Children,2,2,2


In [23]:
children_new['Children Number'].mean()

2.2936507936507935

The average number of children that a senator has is 2 after rounding.

Political Party

In [24]:
all_party = all_senators.loc[all_senators['key'] == 'Political party']
party=all_party.fillna(0)
party.head(10)

,key,value
11,Political party,Democratic
19,Political party,Democratic
26,Political party,Democratic
17,Political party,Democratic
19,Political party,Democratic
23,Political party,Democratic
15,Political party,Democratic
11,Political party,Democratic (1996–present)
6,Political party,Democratic
12,Political party,\nIndependent[a] (1978–present)\nDemocratic[b]...


In [25]:
party['affiliation']= party['value'].str.extract(r"\b(Democratic|Republican)\b")
party.sort_values(by=['value'])

,key,value,affiliation
12,Political party,\nIndependent[a] (1978–present)\nDemocratic[b]...,Democratic
12,Political party,\nIndependent[a] (1978–present)\nDemocratic[b]...,Democratic
11,Political party,Democratic,Democratic
7,Political party,Democratic,Democratic
12,Political party,Democratic,Democratic
...,...,...,...
20,Political party,Republican (1994–present),Republican
10,Political party,Republican (1998–present),Republican
8,Political party,Republican (2010–present),Republican
5,Political party,Republican (2012–present),Republican


In [26]:
affiliation = party.groupby(['affiliation'])['affiliation'].count()
affiliation

affiliation
Democratic    66
Republican    59
Name: affiliation, dtype: int64

There are 66 senators of Democratic party and 59 senators in Republican party.